# Работа с временнЫми данными

## Типизированные массивы с временем: тип `datetime64` в NumPy

NumPy предложил тип `datetime64` для работы со временем:

In [1]:
import numpy as np
date = np.array('2015-07-04', dtype=np.datetime64)
date

array('2015-07-04', dtype='datetime64[D]')

Как только у нас есть дата в формате NumPy, мы можем быстро применить векторные операции:

In [5]:
date + np.arange(12)

array(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
       '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
       '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
      dtype='datetime64[D]')

In [6]:
# Только дата
np.datetime64('2015-07-04')

numpy.datetime64('2015-07-04')

In [7]:
# Дата и время
np.datetime64('2015-07-04 12:00')

numpy.datetime64('2015-07-04T12:00')

In [8]:
# Время с учётом наносекунд
np.datetime64('2015-07-04 12:59:59.50', 'ns')

numpy.datetime64('2015-07-04T12:59:59.500000000')

Доступные в NumPy коды формата времени https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-units

### Даты и время в Pandas: лучшее из двух миров

Pandas имеет дополнительный тип `Timestamp` и `DatetimeIndex` для работы со временем и временнЫми рядами:

In [12]:
import pandas as pd
date = pd.to_datetime("4th of July, 2015")
date

Timestamp('2015-07-04 00:00:00')

In [13]:
date.strftime("%A")

'Saturday'

Также доступны векторные операции:

In [15]:
date + pd.to_timedelta(np.arange(12), 'D')

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)

## Ряды времени в Pandas: индексирование по времени

Мы можем создать объект типа `Series` содержащий индекс по дате:

In [18]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

После того, как у нас есть индекс по датам, мы можем использовать любые типы индексирования, которые мы рассмотрели ранее:

In [20]:
data['2014-07-04':'2015-07-04']

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

In [22]:
# Для получения среза можно передать даже просто год
data["2015"]

2015-07-04    2
2015-08-04    3
dtype: int64

## Структуры данных Pandas для хранения временных радов

Эта секция познакомит с фундаментальными структурами Pandas для работы с временными рядами:

* Для меток времени есть тип `Timestamp` и связанную структуру для индексов `DatetimeIndex`.
* Для периодов времени есть тип `Period` и связанную структуру для индексов `PeriodIndex`.
* Для разниц по времени или продолжительности, есть тип `Timedelta` и соответствующий индекс `TimedeltaIndex`.

Наиболее фундаментальные из них это `Timestamp` и `DatetimeIndex`. Эти классы могут использоваться напрямую, но обычно соответствующие объекты создаются через метод `pd.to_datetime()`. Причём могут использоваться различные типы представления даты:

In [24]:
from datetime import datetime
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
               '2015-07-08'],
              dtype='datetime64[ns]', freq=None)

Любой объект типа `DatetimeIndex` может быть сконвертирован в `PeriodIndex` с помощью метода `to_period()` с возможностью указания частоты.

In [26]:
# Частота установлена в днях - `D`
dates.to_period("D")

PeriodIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
             '2015-07-08'],
            dtype='period[D]')

Объект типа `TimedeltaIndex` создается когда происходит вычитание дат:

In [28]:
dates - dates[0]

TimedeltaIndex(['0 days', '1 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq=None)

### Регулярные последовательности: `pd.date_range()`

Чтобы создание рядов дат было более удобным, в Pandas есть несколько функций для этой цели:

* `pd.date_range()` для меток времени
* `pd.period_range()` для периодов времени
* `pd.timedelta_range()` для разниц времени

Можно указать дату начала, дату окончания и частоту (частота по умолчанию - 1 день):

In [30]:
pd.date_range("2015-07-03", "2015-07-10")

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

Также можно указать начальную дату и количество периодов:

In [32]:
pd.date_range('2015-07-03', periods=8)

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

Частоту можно задать с помощью ключевого слова `freq`. Например далее мы создаём диапазон меток времени с частотой 1 час:

In [34]:
pd.date_range('2015-07-03', periods=8, freq='H')

DatetimeIndex(['2015-07-03 00:00:00', '2015-07-03 01:00:00',
               '2015-07-03 02:00:00', '2015-07-03 03:00:00',
               '2015-07-03 04:00:00', '2015-07-03 05:00:00',
               '2015-07-03 06:00:00', '2015-07-03 07:00:00'],
              dtype='datetime64[ns]', freq='H')

In [37]:
# Ежемесячные периоды
pd.period_range("2015-07", periods=8, freq="M")

PeriodIndex(['2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
             '2016-01', '2016-02'],
            dtype='period[M]')

In [38]:
# Часовые дельты времени
pd.timedelta_range(0, periods=10, freq='H')

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00'],
               dtype='timedelta64[ns]', freq='H')

## Частоты и смещения

Доступны следующие коды частот:

| Code | Description       | Code | Description          |
|------|-------------------|------|----------------------|
| `D`  | Calendar day      | `B`  | Business day         |
| `W`  | Weekly            |      |                      |
| `M`  | Month end         | `BM` | Business month end   |
| `Q`  | Quarter end       | `BQ` | Business quarter end |
| `A`  | Year end          | `BA` | Business year end    |
| `H`  | Hours             | `BH` | Business hours       |
| `T`  | Minutes           |      |                      |
| `S`  | Seconds           |      |                      |
| `L`  | Milliseconds      |      |                      |
| `U`  | Microseconds      |      |                      |
| `N`  | Nanoseconds       |      |                      |

Месячная, квартальная и годовая частота принимаются на конец каждого периода. Путём добавления суффикса `S`, указанные частоты будут приниматься на начало каждого периода:

| Code  | Description       | Code  | Description            |
|-------|-------------------|-------|------------------------|
| `MS`  | Month start       |`BMS`  | Business month start   |
| `QS`  | Quarter start     |`BQS`  | Business quarter start |
| `AS`  | Year start        |`BAS`  | Business year start    |

Также можно изменить месяц начала квартала или года с помощью суффиксов:

- `Q-JAN`, `BQ-FEB`, `QS-MAR`, `BQS-APR`, etc.
- `A-JAN`, `BA-FEB`, `AS-MAR`, `BAS-APR`, etc.

Также и для недель:

- `W-SUN`, `W-MON`, `W-TUE`, `W-WED`, etc.

Поверх всего, коды можно комбинировать для получения уникальных частот. Например, для частоты 2 часа 30 минут, мы можем скомбинировать коды часа `H` и минуты `T`:

In [40]:
pd.timedelta_range(0, periods=9, freq="2H30T")

TimedeltaIndex(['0 days 00:00:00', '0 days 02:30:00', '0 days 05:00:00',
                '0 days 07:30:00', '0 days 10:00:00', '0 days 12:30:00',
                '0 days 15:00:00', '0 days 17:30:00', '0 days 20:00:00'],
               dtype='timedelta64[ns]', freq='150T')

Все эти короткие коды относятся к реальным объектам Pandas, которые могут быть найдены в модуле `pd.tseries.offsets`. Например мы можем напрямую использовать рабочий день:

In [42]:
from pandas.tseries.offsets import BDay
pd.date_range('2015-07-01', periods=5, freq=BDay())

DatetimeIndex(['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-06',
               '2015-07-07'],
              dtype='datetime64[ns]', freq='B')

# Resampling, Shifting and Windowing

https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html#Resampling,-Shifting,-and-Windowing